### 20 Newsgroup 토픽 모델링

**20개 중 8개의 주제 데이터 로드 및 Count기반 피처 벡터화. LDA는 Count기반 Vectorizer만 적용합니다**

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 전자공학, 의학 등 8개 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 category만 추출. featch_20newsgroups( )의 categories에 cats 입력
news_df= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=0)

#LDA 는 Count기반의 Vectorizer만 적용합니다.  
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [25]:
#참고
news_df.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

**LDA 객체 생성 후 Count 피처 벡터화 객체로 LDA수행**

In [8]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

-LDA 하이퍼파라미터 관련 질문) LDA가 동작하는 과정 중에서 모든 단어들이 토픽 할당 분포가 변경되지 않을 때까지 반복적인 수행을 거치기 때문에 이에 관련된 파라미터로 max_iter가 있다고 하셨습니다. 그렇다면 혹시 max_iter를 예를들어 300으로 설정했는데 알고리즘이 동작하다가 200번째에 모든 단어들의 토픽 할당이 수렴이 된다면 그 때 바로 그냥 멈추고 결과를 출력하나요? (마치 XGBoost나 뉴럴네트워크 처럼 early_stopping 기능처럼요)  

-답변) 네 맞습니다. LDA 내부 알고리즘이 EM(Expectation Maximization) step을 반복하는데 수렴이 될 경우 max_iter까지 반복하지 않고 종료됩니다.

**각 토픽 모델링 주제별 단어들의 연관도 확인**  
lda객체의 components_ 속성은 주제별로 개별 단어들의 연관도 정규화 숫자가 들어있음

shape는 주제 개수 X 피처 단어 개수  

components_ 에 들어 있는 숫자값은 각 주제별로 단어가 나타난 횟수를 정규화 하여 나타냄.   

숫자가 클 수록 토픽에서 단어가 차지하는 비중이 높음  

In [10]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.87175875e+02, 3.26401283e+02, 1.67643972e+02, ...,
        7.19452113e+01, 1.25032429e-01, 1.25031801e-01],
       [1.25100534e-01, 1.25200157e-01, 1.25116306e-01, ...,
        5.23417043e+00, 1.25013172e-01, 1.25003054e-01],
       [8.10165562e+01, 1.29756513e+01, 1.58877134e+01, ...,
        2.02263029e+01, 1.25006040e-01, 1.25000074e-01],
       ...,
       [4.47519574e+01, 2.07610664e-01, 4.16627335e+00, ...,
        6.91641165e+00, 1.25006504e-01, 1.25000079e-01],
       [3.40790460e+01, 1.68526498e+01, 1.07933631e+01, ...,
        6.35434267e+01, 1.25001835e-01, 1.25001866e-01],
       [1.25080974e-01, 2.07187470e+02, 1.25006033e-01, ...,
        6.49764265e+01, 2.62124935e+02, 2.49124958e+02]])

**각 토픽별 중심 단어 확인**

In [ ]:
def display_topic_words(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_): # 8개의 토픽이 순서대로 돌아감
        print('\nTopic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1] #[::-1]은 argsort에서 내림차순의 의미
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' + '.join([str(feature_names[i])+'*'+str(round(topic[i],1)) for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출(각 단어마다 정규화된 횟수도 같이 추출됨)
display_topic_words(lda, feature_names, 15)

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 전자공학, 의학 등 8개 주제를 추출. 

**개별 문서별 토픽 분포 확인**

lda객체의 transform()을 수행하면 개별 문서별 토픽 분포를 반환함. 

In [17]:
doc_topics = lda.transform(feat_vect)
print(doc_topics.shape)
print(doc_topics[:3])

(7862, 8)
[[0.01041839 0.55396999 0.01042574 0.38346606 0.01042319 0.01043351
  0.01042806 0.01043505]
 [0.07922454 0.00186778 0.00186834 0.001869   0.00186785 0.79815854
  0.00186917 0.11327478]
 [0.0036816  0.28344141 0.00368174 0.00368224 0.18191133 0.51623794
  0.00368303 0.0036807 ]]


**개별 문서별 토픽 분포도를 출력**

20newsgroup으로 만들어진 문서명을 출력.

fetch_20newsgroups()으로 만들어진 데이터의 filename속성은 모든 문서의 문서명을 가지고 있음.

filename속성은 절대 디렉토리를 가지는 문서명을 가지고 있으므로 '\\'로 분할하여 맨 마지막 두번째 부터 파일명으로 가져옴

In [19]:
def get_filename_list(newsdata):
    filename_list=[]

    for file in newsdata.filenames:
            #print(file) #(file 보면 C:\~~~~\~~\~~ 이런식으로 되어 있음)
            filename_temp = file.split('\\')[-2:] #'(\'는 윈도우에서 '\\' 두개로 써야 함) 
            filename = '.'.join(filename_temp)
            filename_list.append(filename)
    
    return filename_list

filename_list = get_filename_list(news_df)
print("filename 개수:",len(filename_list), "filename list 10개만:",filename_list[:10])

filename 개수: 7862 filename list 10개만: ['soc.religion.christian.20630', 'sci.med.59422', 'comp.graphics.38765', 'comp.graphics.38810', 'sci.med.59449', 'comp.graphics.38461', 'comp.windows.x.66959', 'rec.motorcycles.104487', 'sci.electronics.53875', 'sci.electronics.53617']


**DataFrame으로 생성하여 문서별 토픽 분포도 확인**

In [20]:
import pandas as pd 

topic_names = ['Topic #'+ str(i) for i in range(0, 8)]
doc_topic_df = pd.DataFrame(data=doc_topics, columns=topic_names, index=filename_list)
doc_topic_df.head(20)

,Topic #0,Topic #1,Topic #2,Topic #3,Topic #4,Topic #5,Topic #6,Topic #7
soc.religion.christian.20630,0.010418,0.553970,0.010426,0.383466,0.010423,0.010434,0.010428,0.010435
sci.med.59422,0.079225,0.001868,0.001868,0.001869,0.001868,0.798159,0.001869,0.113275
comp.graphics.38765,0.003682,0.283441,0.003682,0.003682,0.181911,0.516238,0.003683,0.003681
comp.graphics.38810,0.003682,0.182998,0.381892,0.003684,0.098440,0.003683,0.321931,0.003690
sci.med.59449,0.005443,0.005446,0.005441,0.536006,0.005443,0.431327,0.005446,0.005448
comp.graphics.38461,0.004316,0.622716,0.004317,0.208271,0.147426,0.004320,0.004320,0.004315
comp.windows.x.66959,0.228170,0.574088,0.015641,0.015627,0.119577,0.015629,0.015633,0.015633
rec.motorcycles.104487,0.042086,0.003057,0.049479,0.003051,0.003051,0.003053,0.893168,0.003055
sci.electronics.53875,0.005445,0.005439,0.005444,0.005439,0.005438,0.579290,0.388058,0.005446
sci.electronics.53617,0.010432,0.926968,0.010421,0.010435,0.010439,0.010443,0.010441,0.010421
